In [1]:
from matplotlib import pyplot as plt
# linear algebra
import numpy as np 

# data processing
import pandas as pd 

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import svm

#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

# csv file manipulation
import csv

In [8]:
# read training data 
df = pd.read_csv('./data/train.csv')

# read testing data
test_df = pd.read_csv('./data/test.csv')

In [9]:
# remove records with missing durations
df = df.dropna(subset=['duration', 'fare'])
print(df['duration'].isnull().values.sum())

0


In [10]:
df['trip_duration'] = df['duration']- (df['meter_waiting_till_pickup'] + df['meter_waiting'])

In [12]:
print(df['trip_duration'].isnull().values.sum())

0


In [13]:
test_df['trip_duration'] = test_df['duration']- (test_df['meter_waiting_till_pickup'] + test_df['meter_waiting'])
print(test_df['trip_duration'].isnull().values.sum())

0


In [14]:
df['trip_fair'] = df['fare'] -  (df['additional_fare'] + df['meter_waiting_fare'])
test_df['trip_fair'] = test_df['fare'] -  (test_df['additional_fare'] + test_df['meter_waiting_fare'])

print(df['trip_fair'].isnull().values.sum())
print(test_df['trip_fair'].isnull().values.sum())

0
0


In [15]:
df

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label,trip_duration,trip_fair
0,189123628,10.5,834.0,56.0,0.000000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct,714.0,259.820000
1,189125358,10.5,791.0,47.0,0.000000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct,610.0,187.350000
2,189125719,10.5,1087.0,80.0,0.000000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct,946.0,291.140000
3,189127273,10.5,598.0,271.0,15.663800,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct,259.0,56.136200
5,189129552,10.5,3407.0,182.0,0.000000,112.0,11/1/2019 5:38,11/1/2019 6:35,7.13402,79.8969,6.91865,79.8649,1065.02,correct,3113.0,1054.520000
6,189132829,10.5,1246.0,487.0,0.000000,133.0,11/1/2019 6:29,11/1/2019 6:49,6.84371,79.9051,6.85069,79.8624,266.62,correct,626.0,256.120000
7,189135103,10.5,1333.0,295.0,17.198500,212.0,11/1/2019 6:50,11/1/2019 7:12,6.90760,79.9524,6.90634,79.9042,318.05,correct,826.0,290.351500
8,189139296,10.5,360.0,80.0,4.664000,3.0,11/1/2019 7:00,11/1/2019 7:06,7.26706,80.6064,7.27422,80.6124,100.32,correct,277.0,85.156000
9,189138671,10.5,1539.0,588.0,33.986400,43.0,11/1/2019 7:02,11/1/2019 7:28,6.85137,79.9537,6.84779,79.9274,257.89,correct,908.0,213.403600
10,189135814,10.5,26.0,19.0,0.000000,73.0,11/1/2019 7:27,11/1/2019 7:27,7.00355,79.8819,7.13279,79.8868,694.37,incorrect,-66.0,683.870000


In [16]:
#split into data and label
Y = df['label']
X = df.drop(columns=['tripid', 'label', 'pickup_time', 'drop_time'])

In [17]:
tripIds = test_df['tripid']

test_df = test_df.drop(columns=['tripid', 'pickup_time', 'drop_time' ])

X_test = test_df

In [18]:
#Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X)
X_test = sc.transform(X_test)

In [19]:
def write_output(Y_prediction):
    with open('sampleSubmission.csv', 'w', newline='') as csvfile:
        fieldnames = ['tripid', 'prediction']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        line = 0
    
        for w in Y_prediction:
            output = 1
            if (w == 'incorrect'):
                output = 0
            writer.writerow({'tripid': tripIds[line], 'prediction': output})
            line = line+1


In [22]:


def RFClassifier(X_train, y_train, X_test):
   
    #Create the Classifier
    clf=RandomForestClassifier(n_estimators=100)

    #Train the model using the training sets y_pred=clf.predict(X_test)
    clf.fit(X_train,y_train)

    y_pred=clf.predict(X_test)
    
    return y_pred
    

In [23]:
Y_pred = RFClassifier(X_train,Y, X_test)

write_output(Y_pred)